In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [ ]:
!pip install \
  langchain \
  langchain-openai \
  langgraph \
  openai \
  google-search-results \
  python-dotenv \
  aiohttp \
  beautifulsoup4

In [ ]:
import os
import operator
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langgraph.graph import StateGraph, END

# 최신 LLM 사용
from langchain_openai import ChatOpenAI
from langchain.tools import tool

# SerpAPI용 라이브러리
from serpapi import GoogleSearch

# 환경변수 설정 (직접 키 입력 가능)
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로딩 (운영)
load_dotenv()

# .env 파일내에 공백없이 작성
#SERPAPI_API_KEY=9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196

# 환경변수 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["SERPAPI_API_KEY"] = "9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196"

# 웹 검색 툴 함수 정의
@tool
def web_scraper(query: str, num_results: int = 5) -> str:
    """구글에서 검색어에 대한 상위 요약 검색 결과를 반환합니다."""
    try:
        serpapi_key = os.getenv("SERPAPI_API_KEY")
        params = {
            "engine": "google",
            "q": query,
            "hl": "ko",
            "gl": "kr",
            "api_key": serpapi_key
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "organic_results" not in results or not results["organic_results"]:
            return "검색 결과 없음"

        titles = [res.get("title", "제목 없음") for res in results["organic_results"][:num_results]]
        return "\n".join(f"- {title}" for title in titles)
    except Exception as e:
        return f"웹 검색 오류: {str(e)}"

tools = [web_scraper]

# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 유능한 검색 도우미이다. 사용자의 요청에 따라 웹에서 정보를 찾아 요약하라."),
    MessagesPlaceholder(variable_name="messages"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# LLM 정의 (Tool Binding 가능)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Tool Calling Agent 구성
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# LangGraph 상태 정의
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

# 노드 함수 정의
def run_agent_node(state: AgentState) -> AgentState:
    result = agent_executor.invoke({"messages": state["messages"]})
    return {"messages": [result]}

# 그래프 정의 및 컴파일 : LangGraph는 각 노드를 정의하고, 그 노드들 간의 데이터 흐름과 상태 변화를 제어하는 워크플로우 프레임워크이다.
# LangGraph에서 각 노드를 정의하고 이들을 분기하거나 이어 붙일 때, 이전 노드의 처리 결과를 다음 노드에서 처리히게 구성하는 흐름을 반영한다.
graph = StateGraph(AgentState)
graph.add_node("agent", run_agent_node)
graph.set_entry_point("agent")
graph.set_finish_point("agent")

app = graph.compile()

# LangGraph 실행 : 사전에 Agent들 간의 업무 순서대로 정의된 각 Agent 노드를 실행한다.
# 실제로는 Agent가 단순히 Tool Calling만 하는게 아니고, Agent 역할에 맞는 AI 서비스 처리를 하면서 필요 시 Tool Callling을 하게 된다.
input_state = {"messages": [HumanMessage(content="2026 수능 날짜 알려줘")]}
result = app.invoke(input_state)

# 응답 출력
last_msg = result["messages"][-1]
print("🟢 응답:\n", getattr(last_msg, "content", last_msg))




> Entering new AgentExecutor chain...

Invoking: `web_scraper` with `{'query': '2026 수능 날짜'}`


- 2026학년도 대학수학능력시험
- 2026학년도 대학수학능력시험은 2025년 11월 13일(목) ...
- [그래픽] 2026학년도 수능 주요 일정
- [2026 대입, 07수능] 07년생 수능일자 및 주요대입일정 ... - 블로그
- 수능 디데이 카운터 : 2026학년도 수능일 - superkts.com2026학년도 대학수학능력시험(수능)은 2025년 11월 13일 목요일에 실시됩니다.

> Finished chain.
🟢 응답:
 {'messages': [HumanMessage(content='2026 수능 날짜 알려줘', additional_kwargs={}, response_metadata={})], 'output': '2026학년도 대학수학능력시험(수능)은 2025년 11월 13일 목요일에 실시됩니다.'}
